In [ ]:
from keras import models

In [ ]:
model = load_model('model_best.h5')
model.summary

In [ ]:
# 이미지 제너레이터로 학습된 모델의 원핫인코딩 레이블은 어떻게 되는지 확인하기
# 이것을 확인해야 나중에 디코딩으로 각각의 위치가 어떤 종을 나타내는지 알수 있고 이를 바탕으로 그래프를 그릴 수 있음있음